# ME Woodroof Research Area Group, Themes Classification

In [33]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import time
import re

from openai import OpenAI

In [34]:
# Loading "Sheet 2" from the "5 Year ME Award Trend by PI Org_8.29.24 - Copy.xlsx" to view project titles
award_data = pd.read_excel(r'C:\Users\donav\OneDrive\Desktop\Gatech\Research\GRA Ford Research\ME_Woodruff_Classifier\data\5 Year ME Award Trend by PI Org_8.29.24 - Copy.xlsx', sheet_name='Sheet2')


In [5]:
award_data.head()  # Display the first few rows of the dataframe

,Award Type Desc,Co Pdpi Names,Contract Type Desc,Department,Doch Keywords,FY Year,Pdpi Name,Pdpi Org Name,Proj Status Desc,Proj Title,RAG,Theme,Project Type Desc,Ps Project Number,Sponsor Business Type Desc,Sponsor Name,Wd Award Id,Award Funded Amount
0,NaN,NaN,Cost Reimbursement No Fee,Mechanical Engineering,[WEARABLE ELECTRONICS],FY21,"YEO, WOONHONG",Mechanical Engineering,NaN,CAREER:Interfacial Tissue Physics and Wearable...,Bioengineering,Health,NaN,NaN,Government Sponsors,NATIONAL SCIENCE FOUNDATION (NSF)/GENERAL,NaN,89989.00
1,Agreement,"JANG, SEUNG SOON",Cost Plus Fixed Fee,Mechanical Engineering,NaN,FY20,"LEE, SEUNG WOO",Mechanical Engineering,Closeout,DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT...,"Heat Transfer, Combustion, Energy Systems",Sustainability & Energy,Research,GR10026431,Coll/Univ/Res Institutes,KOREAN INSTITUTE OF SCIENCE & TECHNOLOGY INFO/...,AWD-102945,366107.00
2,Agreement,NaN,Cost Reimbursement No Fee,Institue Of Materials,NaN,FY20,"NEU, RICHARD",Mechanical Engineering,Terminated,SSA WEAV3D INC.,NaN,NaN,Research,GR00002520,Industrial Sponsors,"WEAV3D INC/NORCROSS,GA",AWD-000511,1189.00
3,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BAIR, SCOTT",Mechanical Engineering,Terminated,CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER,Tribology,"Design, Materials, Manufacturing",Research,GR00003524,Coll/Univ/Res Institutes,"UNIVERSITY OF KAISERSLAUTERN/KAISERSLAUTERN, G...",AWD-000825,134499.94
4,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BAIR, SCOTT",Mechanical Engineering,Terminated,DEVELOPMENT OF A HIGH PRESSURE VISCOMETER (COO...,Tribology,"Design, Materials, Manufacturing",Research,GR00003346,Coll/Univ/Res Institutes,"LULEA UNIVERSITY OF TECHNOLOGY/LULEA, SWEDEN",AWD-000778,132133.03


In [8]:
award_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Award Type Desc             1132 non-null   object 
 1   Co Pdpi Names               238 non-null    object 
 2   Contract Type Desc          1128 non-null   object 
 3   Department                  1133 non-null   object 
 4   Doch Keywords               302 non-null    object 
 5   FY Year                     1133 non-null   object 
 6   Pdpi Name                   1133 non-null   object 
 7   Pdpi Org Name               1133 non-null   object 
 8   Proj Status Desc            1132 non-null   object 
 9   Proj Title                  1133 non-null   object 
 10  RAG                         15 non-null     object 
 11  Theme                       15 non-null     object 
 12  Project Type Desc           1132 non-null   object 
 13  Ps Project Number           1129 

In [33]:
# Output a string of all the project titles
project_titles = award_data['Proj Title'].tolist()


In [34]:
print(project_titles)

['CAREER:Interfacial Tissue Physics and Wearable Scalp Technologies for Long-', 'DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT FUEL CELL MEA OPERATED WITH', 'SSA WEAV3D INC.', 'CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER', 'DEVELOPMENT OF A HIGH PRESSURE VISCOMETER (COOPERATIVE AGREEMENT)', 'BASIC SCIENCE OF RADIATION EFFECTS IN FERROELECTRIC MULTI-FUNCTIONAL MEMS..', 'GUIDEWIRE IVUS DEVELOPMENT', 'NON-DESTRUCTIVE TESTING OF ADDITIVE MANUFACTURED AEROSPACE COMPONENTS', 'STTR: SMALLSAT STIRLING CRYOCOOLER/ STTR TOPIC MDA17-T003', 'FABRICATION AND EVALUATION OF MICROFABRICATED VAPOR PRECONCENTRATION', 'ADVANCED POWER ELECTRONICS THERMAL MANAGEMENT', 'EMCOOL GRA FUNDING JUNE 2019', 'RECHARGE, RE-LIQUIFY, RE-WET (RE3):  SELF-HEALING INTERFACES FOR SOLID...', 'REAL TIME MICROFLUIDIC SORTING TO EVALUATE DRUG RESISTANCE IN LEUKEMIA ...', 'GENERATION II REINVENTED TOILET', 'Fabrication and evaluation of flexible graphene-polymer', 'UTRATHIN, CONFORTABLE, MULTIFUNCTIONAL BIOPATH FOR PORTABLE 

In [35]:
# Only keep columns Pdpi Name, Proj Title
award_projects_data = award_data

In [36]:
# Remove any rows wiht missing values in the 'Proj Title' column
award_projects_data = award_projects_data.dropna(subset=['Proj Title'])

In [42]:
award_projects_data.head()  # Display the first few rows of the cleaned dataframe

,Award Type Desc,Co Pdpi Names,Contract Type Desc,Department,Doch Keywords,FY Year,Pdpi Name,Pdpi Org Name,Proj Status Desc,Proj Title,RAG,Theme,Project Type Desc,Ps Project Number,Sponsor Business Type Desc,Sponsor Name,Wd Award Id,Award Funded Amount
0,NaN,NaN,Cost Reimbursement No Fee,Mechanical Engineering,[WEARABLE ELECTRONICS],FY21,"YEO, WOONHONG",Mechanical Engineering,NaN,CAREER:Interfacial Tissue Physics and Wearable...,Bioengineering,Health,NaN,NaN,Government Sponsors,NATIONAL SCIENCE FOUNDATION (NSF)/GENERAL,NaN,89989.00
1,Agreement,"JANG, SEUNG SOON",Cost Plus Fixed Fee,Mechanical Engineering,NaN,FY20,"LEE, SEUNG WOO",Mechanical Engineering,Closeout,DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT...,"Heat Transfer, Combustion, Energy Systems",Sustainability & Energy,Research,GR10026431,Coll/Univ/Res Institutes,KOREAN INSTITUTE OF SCIENCE & TECHNOLOGY INFO/...,AWD-102945,366107.00
2,Agreement,NaN,Cost Reimbursement No Fee,Institue Of Materials,NaN,FY20,"NEU, RICHARD",Mechanical Engineering,Terminated,SSA WEAV3D INC.,NaN,NaN,Research,GR00002520,Industrial Sponsors,"WEAV3D INC/NORCROSS,GA",AWD-000511,1189.00
3,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BAIR, SCOTT",Mechanical Engineering,Terminated,CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER,Tribology,"Design, Materials, Manufacturing",Research,GR00003524,Coll/Univ/Res Institutes,"UNIVERSITY OF KAISERSLAUTERN/KAISERSLAUTERN, G...",AWD-000825,134499.94
4,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BAIR, SCOTT",Mechanical Engineering,Terminated,DEVELOPMENT OF A HIGH PRESSURE VISCOMETER (COO...,Tribology,"Design, Materials, Manufacturing",Research,GR00003346,Coll/Univ/Res Institutes,"LULEA UNIVERSITY OF TECHNOLOGY/LULEA, SWEDEN",AWD-000778,132133.03


In [10]:
# Unique values in depatment column
unique_departments = award_projects_data['Department'].unique()
print(unique_departments)

['Mechanical Engineering' 'Institue Of Materials'
 'Renewable Bioproducts Institute' 'School Of Industrial Design'
 'Georgia Tech Manufacturing Institute' 'College Of Design'
 'Advanced Concept Lab - Gtri' 'Strategic Energy Initiative'
 'Materials Science And Engr' 'Bioengineering & Bioscience'
 'Biomedical Engineering/gt Emory'
 'Institute For Electronics And Nanotechnology']


In [37]:
award_projects_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1133 entries, 0 to 1132
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Award Type Desc             1132 non-null   object 
 1   Co Pdpi Names               238 non-null    object 
 2   Contract Type Desc          1128 non-null   object 
 3   Department                  1133 non-null   object 
 4   Doch Keywords               302 non-null    object 
 5   FY Year                     1133 non-null   object 
 6   Pdpi Name                   1133 non-null   object 
 7   Pdpi Org Name               1133 non-null   object 
 8   Proj Status Desc            1132 non-null   object 
 9   Proj Title                  1133 non-null   object 
 10  RAG                         15 non-null     object 
 11  Theme                       15 non-null     object 
 12  Project Type Desc           1132 non-null   object 
 13  Ps Project Number           1129 non-n

In [38]:
# Function to reformat professor names
def reformat_name(name):
    parts = name.split(", ")
    if len(parts) == 2:
        last_name, first_middle_name = parts[0], parts[1]
        return f"{first_middle_name} {last_name}"
    return name

In [39]:
# Apply reformat_name function to the Pdpi Name column
award_projects_data['Formatted Name'] = award_projects_data['Pdpi Name'].apply(reformat_name)

C:\Users\donav\AppData\Local\Temp\ipykernel_12824\1503428992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  award_projects_data['Formatted Name'] = award_projects_data['Pdpi Name'].apply(reformat_name)


In [45]:
# drop the RAG and Theme columns
award_projects_data = award_projects_data.drop(columns=['RAG', 'Theme'])

# Approach 1:
- Classification utilizing ChatGPT and prompt engineering

In [ ]:
#Project Key
client = OpenAI(
    # This is the default and can be omitted
    api_key=PLACEHOLDER
)

In [19]:
# Function to classify RAG and themes
def classify_title(title):
    # Define the prompt for the OpenAI API
    prompt = f"""
    Classify the following project title into one Research Area Group (RAG) and suggest relevant themes. 
    \n\nTitle: '{title}'
    
    Research Area Groups (choose ONLY ONE):
    - Acoustics/Dynamics
    - AI and Informatics for ME (AI2ME)
    - Automation, Robotics, and Control
    - Bioengineering
    - CAE and Design
    - Engineering Education
    - Fluid Mechanics
    - Heat Transfer, Combustion, & Energy Systems
    - Manufacturing
    - Mechanics of Materials
    - Medical Physics
    - Micro and Nano Engineering
    - Nuclear & Radiological Engineering
    - Tribology
    
    Themes (choose 1-3 from this list ONLY):
    - Sustainability & Energy
    - Health
    - Water & Agriculture
    - Ocean, Space, Aerospace
    - Design, Materials, Manufacturing
    - Mobility & Automotive
    - Robotics
    
    Please follow these conditions:
    - Select ONLY ONE option for the RAG.
    - Select one to three options (MAX 3) for the themes.
    - Use only the options provided; do not add or invent any new categories.
    
    Response format:
    - RAG: <One Research Area Group>
    - Themes: [<Theme1>, <Theme2>, <Theme3> (if applicable)]
    """

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
    )
    
    # Extract response content
    content = response.choices[0].message.content

    print(content)  # Print the response content for debugging
    
    # Parse the RAG and Themes from the response content using regex
    rag_match = re.search(r'RAG:\s*(.*)', content)
    themes_match = re.search(r'Themes:\s*(.*)', content)

    # Extract RAG and Themes if they match, or set to None if not found
    rag = rag_match.group(1).strip() if rag_match else None
    themes = themes_match.group(1).strip() if themes_match else None
    
    return rag, themes

In [65]:
# Loop through each row in award_projects_data and apply the classification function
for idx, row in award_projects_data.iterrows():
    try:
        print(f"Processing row {idx + 1}/{len(award_projects_data)}")
        print(f"Project Title: {row['Proj Title']}")
        
        # Classify project title using the ChatGPT API
        rag, themes = classify_title(row['Proj Title'])
        
        # Print and store the classification result
        print(f"RAG: {rag}, Themes: {themes}")
        award_projects_data.loc[idx, 'RAG'] = rag
        award_projects_data.loc[idx, 'Themes'] = themes
        
        # Delay to avoid rate limiting
        time.sleep(1)
    
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        award_projects_data.loc[idx, 'RAG'] = None
        award_projects_data.loc[idx, 'Themes'] = None
        time.sleep(1)

print("\nClassification completed for all rows.")
print(award_projects_data[['Pdpi Name', 'Proj Title', 'RAG', 'Themes']].head())

Processing row 1/1133
Project Title: CAREER:Interfacial Tissue Physics and Wearable Scalp Technologies for Long-
- RAG: Bioengineering
- Themes: [Health, Water & Agriculture]
RAG: Bioengineering, Themes: [Health, Water & Agriculture]
Processing row 2/1133
Project Title: DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT FUEL CELL MEA OPERATED WITH
- RAG: Heat Transfer, Combustion, & Energy Systems
- Themes: [Sustainability & Energy]
RAG: Heat Transfer, Combustion, & Energy Systems, Themes: [Sustainability & Energy]
Processing row 3/1133
Project Title: SSA WEAV3D INC.
- RAG: CAE and Design
- Themes: [Design, Materials, Manufacturing]
RAG: CAE and Design, Themes: [Design, Materials, Manufacturing]
Processing row 4/1133
Project Title: CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER
- RAG: Fluid Mechanics
- Themes: [Health, Water & Agriculture]
RAG: Fluid Mechanics, Themes: [Health, Water & Agriculture]
Processing row 5/1133
Project Title: DEVELOPMENT OF A HIGH PRESSURE VISCOMETER (COOPERATIVE AG

In [24]:
# I want to join the RAG and Themes column into the main award_data dataframe on the Proj Title column
merged_award_data = award_data.merge(award_projects_data[['Proj Title', 'RAG', 'Themes']], on='Proj Title', how='left')  # Merge on 'Proj Title'

In [29]:
merged_award_data.info()  # Check the info of the merged dataframe

<class 'pandas.core.frame.DataFrame'>
Index: 1300 entries, 30 to 2356
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Award Type Desc             1296 non-null   object 
 1   Co Pdpi Names               283 non-null    object 
 2   Contract Type Desc          1292 non-null   object 
 3   Department                  1297 non-null   object 
 4   Doch Keywords               349 non-null    object 
 5   FY Year                     1297 non-null   object 
 6   Pdpi Name                   1297 non-null   object 
 7   Pdpi Org Name               1297 non-null   object 
 8   Proj Status Desc            1296 non-null   object 
 9   Proj Title                  1297 non-null   object 
 10  RAG_x                       17 non-null     object 
 11  Theme                       17 non-null     object 
 12  Project Type Desc           1296 non-null   object 
 13  Ps Project Number           1292 non-

In [27]:
# sort the merged dataframe by PDPi Name and FY Year
merged_award_data = merged_award_data.sort_values(by=['Pdpi Name', 'FY Year'])
merged_award_data.head(10)

,Award Type Desc,Co Pdpi Names,Contract Type Desc,Department,Doch Keywords,FY Year,Pdpi Name,Pdpi Org Name,Proj Status Desc,Proj Title,RAG_x,Theme,Project Type Desc,Ps Project Number,Sponsor Business Type Desc,Sponsor Name,Wd Award Id,Award Funded Amount,RAG_y,Themes
30,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Active,NEXT GENERATION PAPER MACHINE - MULTIPHASE FOR...,NaN,NaN,Research,GR00003789,Industrial Sponsors,KIMBERLY CLARK CORP/GA,AWD-000848,200000.0,Manufacturing,"[Sustainability & Energy, Design, Materials, M..."
31,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Active,NEXT GENERATION PAPER MACHINE - MULTIPHASE FOR...,NaN,NaN,Research,GR00003789,Industrial Sponsors,KIMBERLY CLARK CORP/GA,AWD-000848,200000.0,Manufacturing,"[Sustainability & Energy, Design, Materials, M..."
32,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Active,NEXT GENERATION PAPER MACHINE - MULTIPHASE FOR...,NaN,NaN,Research,GR00003789,Industrial Sponsors,KIMBERLY CLARK CORP/GA,AWD-000848,200000.0,Manufacturing,"[Sustainability & Energy, Design, Materials, M..."
33,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Active,NEXT GENERATION PAPER MACHINE-MULTIPHASE FORMING,NaN,NaN,Research,GR00003311,Industrial Sponsors,"SOLENIS/WILMINGTON, DE",AWD-000753,200000.0,Manufacturing,"[Design, Materials, Manufacturing]"
34,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Closeout,NEXT GENERATION PAPER MACHINE - MULTIPHASE FOR...,NaN,NaN,Research,GR00002724,Industrial Sponsors,INTERNATIONAL PAPER CO,AWD-000553,200000.0,Manufacturing,"[Sustainability & Energy, Design, Materials, M..."
35,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Closeout,NEXT GENERATION PAPER MACHINE - MULTIPHASE FOR...,NaN,NaN,Research,GR00002724,Industrial Sponsors,INTERNATIONAL PAPER CO,AWD-000553,200000.0,Manufacturing,"[Sustainability & Energy, Design, Materials, M..."
36,Agreement,NaN,Cost Reimbursement No Fee,Renewable Bioproducts Institute,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Closeout,NEXT GENERATION PAPER MACHINE - MULTIPHASE FOR...,NaN,NaN,Research,GR00002724,Industrial Sponsors,INTERNATIONAL PAPER CO,AWD-000553,200000.0,Manufacturing,"[Sustainability & Energy, Design, Materials, M..."
209,Agreement,"EGERTSDOTTER, ELSE-MARIE ULRIKA",Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Closeout,TISSUE CULTURE FOR THE 21ST CENTURY- SYSTEM AU...,NaN,NaN,Research,GR00003778,Industrial Sponsors,"SCION/ ROTORUA, NEW ZEALAND",AWD-000900,199059.0,Bioengineering,"[Health, Water & Agriculture]"
210,Agreement,"EGERTSDOTTER, ELSE-MARIE ULRIKA",Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Closeout,TISSUE CULTURE FOR THE 21ST CENTURY- SYSTEM AU...,NaN,NaN,Research,GR00003778,Industrial Sponsors,"SCION/ ROTORUA, NEW ZEALAND",AWD-000900,199059.0,Bioengineering,"[Health, Water & Agriculture]"
211,Agreement,"EGERTSDOTTER, ELSE-MARIE ULRIKA",Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"AIDUN, CYRUS",Mechanical Engineering,Closeout,TISSUE CULTURE FOR THE 21ST CENTURY- SYSTEM AU...,NaN,NaN,Research,GR00003778,Industrial Sponsors,"SCION/ ROTORUA, NEW ZEALAND",AWD-000900,199059.0,Bioengineering,"[Health, Water & Agriculture]"


In [28]:
# Remove any duplicate rows
merged_award_data = merged_award_data.drop_duplicates()

In [75]:
# Save the updated dataframe to a new excel file
output_file_path = r'C:\Users\donav\OneDrive\Desktop\Gatech\Research\GRA Ford Research\ME_Woodroof_Themes_Clasification\data\award_data_with_classification_2.xlsx'

award_data.to_excel(output_file_path, index=False)

# Approach 2:
- Same approach but give the agent more information (Project Title, Professor Name, and Department)

In [ ]:
# Updated classification to use professor name and department as well
def classify_title(professor_name, department, title):
    messages = [
        {"role": "system", "content": "You are a helpful assistant. You are classifying Georgia Tech research projects based on their titles, professors, and departments."},
        {"role": "user", "content": f"""
        Classify the following project title into one Research Area Group (RAG) and suggest relevant themes. 

        Project Information:
        - Professor: {professor_name}
        - Department: {department}
        - Title: '{title}'
        
        Research Area Groups (choose ONLY ONE):
        - Acoustics/Dynamics
        - AI and Informatics for ME (AI2ME)
        - Automation, Robotics, and Control
        - Bioengineering
        - CAE and Design
        - Engineering Education
        - Fluid Mechanics
        - Heat Transfer, Combustion, & Energy Systems
        - Manufacturing
        - Mechanics of Materials
        - Medical Physics
        - Micro and Nano Engineering
        - Nuclear & Radiological Engineering
        - Tribology
        
        Themes (choose ONLY ONE):
        - Sustainability & Energy
        - Health, Water & Agriculture
        - Ocean, Space, Aerospace
        - Design, Materials, Manufacturing
        - Mobility/Automotive/Robotics
        
        Please follow these conditions:
        - Select ONLY ONE option for the RAG.
        - Select ONLY ONE option for the themes.
        - Use only the options provided; do not add or invent any new categories.
        
        Response format:
        - RAG: <One Research Area Group>
        - Themes: <One Theme>
        """}
    ]
    
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    # Extract response content
    content = response.choices[0].message.content

    print(content)  # Print the response content for debugging
    
    # Parse the RAG and Themes from the response content using regex
    rag_match = re.search(r'RAG:\s*(.*)', content)
    themes_match = re.search(r'Themes:\s*(.*)', content)

    # Extract RAG and Themes if they match, or set to None if not found
    rag = rag_match.group(1).strip() if rag_match else None
    themes = themes_match.group(1).strip() if themes_match else None
    
    return rag, themes

In [ ]:
#  Loop through each row in award_projects_data and apply the classification function
for idx, row in award_projects_data.iterrows():
    try:
        print(f"Processing row {idx + 1}/{len(award_projects_data)}")
        print(f"Project Title: {row['Proj Title']}")

        # Classify project using professor name, department, and project title
        rag, themes = classify_title(row['Formatted Name'], row['Department'], row['Proj Title'])
        
        # Print and store the classification result
        print(f"RAG: {rag}, Themes: {themes}")
        award_projects_data.loc[idx, 'RAG'] = rag
        award_projects_data.loc[idx, 'Themes'] = themes
        
        # Delay to avoid rate limiting
        time.sleep(1)
    
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        award_projects_data.loc[idx, 'RAG'] = None
        award_projects_data.loc[idx, 'Themes'] = None
        time.sleep(1)

print("\nClassification completed for all rows.")

Processing row 1/1133
Project Title: CAREER:Interfacial Tissue Physics and Wearable Scalp Technologies for Long-
- RAG: Bioengineering
- Themes: [Health, Water & Agriculture]
RAG: Bioengineering, Themes: [Health, Water & Agriculture]
Processing row 2/1133
Project Title: DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT FUEL CELL MEA OPERATED WITH
- RAG: Heat Transfer, Combustion, & Energy Systems
- Themes: [Sustainability & Energy]
RAG: Heat Transfer, Combustion, & Energy Systems, Themes: [Sustainability & Energy]
Processing row 3/1133
Project Title: SSA WEAV3D INC.
- RAG: Manufacturing
- Themes: [Design, Materials, Manufacturing]
RAG: Manufacturing, Themes: [Design, Materials, Manufacturing]
Processing row 4/1133
Project Title: CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER
- RAG: Fluid Mechanics
- Themes: [Sustainability & Energy, Health, Water & Agriculture]
RAG: Fluid Mechanics, Themes: [Sustainability & Energy, Health, Water & Agriculture]
Processing row 5/1133
Project Title: DEVELOPME

In [ ]:
# Drop the formated name column
award_projects_data = award_projects_data.drop(columns=['Formatted Name'])

In [ ]:
award_projects_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1133 entries, 0 to 1132
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Award Type Desc             1132 non-null   object 
 1   Co Pdpi Names               238 non-null    object 
 2   Contract Type Desc          1128 non-null   object 
 3   Department                  1133 non-null   object 
 4   Doch Keywords               302 non-null    object 
 5   FY Year                     1133 non-null   object 
 6   Pdpi Name                   1133 non-null   object 
 7   Pdpi Org Name               1133 non-null   object 
 8   Proj Status Desc            1132 non-null   object 
 9   Proj Title                  1133 non-null   object 
 10  Project Type Desc           1132 non-null   object 
 11  Ps Project Number           1129 non-null   object 
 12  Sponsor Business Type Desc  1133 non-null   object 
 13  Sponsor Name                1133 non-n

In [ ]:
award_projects_data.head(10)

,Award Type Desc,Co Pdpi Names,Contract Type Desc,Department,Doch Keywords,FY Year,Pdpi Name,Pdpi Org Name,Proj Status Desc,Proj Title,Project Type Desc,Ps Project Number,Sponsor Business Type Desc,Sponsor Name,Wd Award Id,Award Funded Amount,RAG,Themes
0,NaN,NaN,Cost Reimbursement No Fee,Mechanical Engineering,[WEARABLE ELECTRONICS],FY21,"YEO, WOONHONG",Mechanical Engineering,NaN,CAREER:Interfacial Tissue Physics and Wearable...,NaN,NaN,Government Sponsors,NATIONAL SCIENCE FOUNDATION (NSF)/GENERAL,NaN,89989.00,Bioengineering,"[Health, Water & Agriculture]"
1,Agreement,"JANG, SEUNG SOON",Cost Plus Fixed Fee,Mechanical Engineering,NaN,FY20,"LEE, SEUNG WOO",Mechanical Engineering,Closeout,DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT...,Research,GR10026431,Coll/Univ/Res Institutes,KOREAN INSTITUTE OF SCIENCE & TECHNOLOGY INFO/...,AWD-102945,366107.00,"Heat Transfer, Combustion, & Energy Systems",[Sustainability & Energy]
2,Agreement,NaN,Cost Reimbursement No Fee,Institue Of Materials,NaN,FY20,"NEU, RICHARD",Mechanical Engineering,Terminated,SSA WEAV3D INC.,Research,GR00002520,Industrial Sponsors,"WEAV3D INC/NORCROSS,GA",AWD-000511,1189.00,Manufacturing,"[Design, Materials, Manufacturing]"
3,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BAIR, SCOTT",Mechanical Engineering,Terminated,CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER,Research,GR00003524,Coll/Univ/Res Institutes,"UNIVERSITY OF KAISERSLAUTERN/KAISERSLAUTERN, G...",AWD-000825,134499.94,Fluid Mechanics,"[Sustainability & Energy, Health, Water & Agri..."
4,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BAIR, SCOTT",Mechanical Engineering,Terminated,DEVELOPMENT OF A HIGH PRESSURE VISCOMETER (COO...,Research,GR00003346,Coll/Univ/Res Institutes,"LULEA UNIVERSITY OF TECHNOLOGY/LULEA, SWEDEN",AWD-000778,132133.03,Fluid Mechanics,"[Design, Materials, Manufacturing]"
5,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"BASSIRI-GHARB, NAZANIN",Mechanical Engineering,Closeout,BASIC SCIENCE OF RADIATION EFFECTS IN FERROELE...,Research,GR10006287,Government Sponsors,DOD/DEFENSE THREAT REDUCTION AGENCY(DTRA)/FT. ...,AWD-100560,347793.00,Nuclear & Radiological Engineering,"[Design, Materials, Manufacturing]"
6,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"DEGERTEKIN, F",Mechanical Engineering,Closeout,GUIDEWIRE IVUS DEVELOPMENT,Research,GR00004228,Industrial Sponsors,"ASAHI INTECC CO LTD/AICHI, JAPAN",AWD-000998,1228000.00,Bioengineering,"[Health, Water & Agriculture]"
7,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"ERTURK, ALPER",Mechanical Engineering,Terminated,NON-DESTRUCTIVE TESTING OF ADDITIVE MANUFACTUR...,Research,GR00002339,Industrial Sponsors,"TRONOS AVIATION CONSULTING/SLEMON PARK PEI, CA...",AWD-000477,107000.00,Manufacturing,"[Design, Materials, Manufacturing, Ocean, Spac..."
8,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"GHIAASIAAN, SEYED",Mechanical Engineering,Closeout,STTR: SMALLSAT STIRLING CRYOCOOLER/ STTR TOPIC...,Research,GR10009414,Industrial Sponsors,"WEST COAST SOLUTIONS/HUNTINGTON BEACH, CA",AWD-101995,1500.00,"Heat Transfer, Combustion, & Energy Systems","[Sustainability & Energy, Ocean, Space, Aerosp..."
9,Agreement,NaN,Cost Reimbursement No Fee,Mechanical Engineering,NaN,FY20,"HESKETH, PETER",Mechanical Engineering,Closeout,FABRICATION AND EVALUATION OF MICROFABRICATED ...,Research,GR00005132,Industrial Sponsors,"MASSTECH INC./COLUMBIA,MD",AWD-001175,79265.00,Micro and Nano Engineering,"[Design, Materials, Manufacturing]"


In [54]:
# print out the unique values of RAG and Themes
print(award_projects_data['RAG'].unique())
print(award_projects_data['Themes'].unique())

['Bioengineering' 'Heat Transfer, Combustion, & Energy Systems'
 'Manufacturing' 'Fluid Mechanics' 'Nuclear & Radiological Engineering'
 'Micro and Nano Engineering' 'Mechanics of Materials'
 'Automation, Robotics, and Control' 'CAE and Design'
 'Engineering Education' 'AI and Informatics for ME (AI2ME)' 'Tribology'
 'Health, Water & Agriculture' 'Mobility/Automotive/Robotics'
 'Medical Physics' 'Ocean, Space, Aerospace' 'Acoustics/Dynamics'
 'Design, Materials, Manufacturing']
['[Health, Water & Agriculture]' '[Sustainability & Energy]'
 '[Design, Materials, Manufacturing]'
 '[Sustainability & Energy, Health, Water & Agriculture]'
 '[Design, Materials, Manufacturing, Ocean, Space, Aerospace]'
 '[Sustainability & Energy, Ocean, Space, Aerospace]'
 '[Health, Water & Agriculture, Sustainability & Energy]'
 '[Sustainability & Energy, Design, Materials, Manufacturing]'
 '[Health, Water & Agriculture, Design, Materials, Manufacturing]'
 '[Mobility/Automotive/Robotics]'
 '[Design, Materials,

In [13]:
# Correct RAGs
rag_selection = [
    'Bioengineering', 
    'Heat Transfer, Combustion, & Energy Systems',
    'Manufacturing', 
    'Fluid Mechanics', 
    'Nuclear & Radiological Engineering',
    'Micro and Nano Engineering', 
    'Mechanics of Materials',
    'Automation, Robotics, and Control', 
    'CAE and Design',
    'Engineering Education', 
    'AI and Informatics for ME (AI2ME)', 
    'Tribology',
    'Medical Physics',  
    'Acoustics/Dynamics'
]

In [14]:
# Create a df to store any values that are not in the rag_selection list
incorrect_rags = award_projects_data[~award_projects_data['RAG'].isin(rag_selection)]

In [23]:
# What are the RAG values in the incorrect_rags dataframe

print(incorrect_rags['RAG'].unique())

['Health, Water & Agriculture' 'Mobility/Automotive/Robotics'
 'Ocean, Space, Aerospace' 'Design, Materials, Manufacturing']


In [17]:
# Count the length of the incorrect_rags dataframe
len(incorrect_rags)

8

In [22]:
# for the rows in incorrect_rags, run the classification function again
for idx, row in incorrect_rags.iterrows():
    try:
        print(f"Processing row {idx + 1}/{len(incorrect_rags)}")
        print(f"Project Title: {row['Proj Title']}")
        
        # Classify project using professor name, department, and project title
        rag, themes = classify_title(row['Pdpi Name'], row['Department'], row['Proj Title'])
        
        # Print and store the classification result
        print(f"RAG: {rag}, Themes: {themes}")
        award_projects_data.loc[idx, 'RAG'] = rag
        award_projects_data.loc[idx, 'Themes'] = themes
        
        # Delay to avoid rate limiting
        time.sleep(1)
    
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        award_projects_data.loc[idx, 'RAG'] = None
        award_projects_data.loc[idx, 'Themes'] = None
        time.sleep(1)

Processing row 60/8
Project Title: GENERATION II REINVENTED TOILET PHASE III
- RAG: Manufacturing
- Themes: [Sustainability & Energy, Health, Water & Agriculture, Design, Materials, Manufacturing]
RAG: Manufacturing, Themes: [Sustainability & Energy, Health, Water & Agriculture, Design, Materials, Manufacturing]
Processing row 78/8
Project Title: ECOCAR 5 - ADVANCED VEHICLE TECHNOLOGY COMPETITION (AVTC) 13
- RAG: Mobility/Automotive/Robotics
- Themes: [Sustainability & Energy, Mobility/Automotive/Robotics]
RAG: Mobility/Automotive/Robotics, Themes: [Sustainability & Energy, Mobility/Automotive/Robotics]
Processing row 178/8
Project Title: SIRIUS
- RAG: Automation, Robotics, and Control
- Themes: [Ocean, Space, Aerospace]
RAG: Automation, Robotics, and Control, Themes: [Ocean, Space, Aerospace]
Processing row 361/8
Project Title: V2X impact on Electric Vehicle Batteries
- RAG: Mobility/Automotive/Robotics
- Themes: [Sustainability & Energy, Mobility/Automotive/Robotics]
RAG: Mobility/Au

In [12]:
# load award_data_with_classification_2.xlsx
award_projects_data = pd.read_excel(r'C:\Users\donav\OneDrive\Desktop\Gatech\Research\GRA Ford Research\ME_Woodruff_Classifier\data\award_data_with_classification_2.xlsx')

In [ ]:
# Save the updated dataframe to a new excel file
output_file_path = r'C:\Users\donav\OneDrive\Desktop\Gatech\Research\GRA Ford Research\ME_Woodruff_Classifier\data\award_data_with_classification_2.xlsx'

award_projects_data.to_excel(output_file_path, index=False)

# Attempt 3: Use RAGs from ME website
- Utilize the Woodruff ME Website to identify which RAGs certain professors are part of. Use this info to make a more informed decision
- Only allow for ONE theme per classification

In [28]:
acoustics_dynamics_faculty = [
    "Costas Arvanitis",
    "Kenneth Cunefare",
    "Nico Declercq",
    "Alper Erturk",
    "Aldo Ferri",
    "François Guillot",
    "Laurence Jacobs",
    "Michael Leamy",
    "Julien Meaud",
    "Karim Sabra",
    "Ji-Xun Zhou",
    "Ben Zinn"
]

ai2me_faculty = [
    "Alexander Alexeev",
    "Nazanin Bassiri-Gharb",
    "Chaitanya Deo",
    "Shaheen Dewji",
    "Andrei Fedorov",
    "Samuel Graham",
    "Itzhak Green",
    "Suhas Jain",
    "Amit Jariwala",
    "Roger Jiao",
    "Yogendra Joshi",
    "Surya Kalidindi",
    "Satish Kumar",
    "Thomas Kurfess",
    "Anirban Mazumdar",
    "David McDowell",
    "Richard Neu",
    "Devesh Ranjan",
    "Christopher Saldaña",
    "Aaron Stebner",
    "Yan Wang",
    "Levi Wood",
    "Shuman Xia",
    "Fan Zhang",
    "Ye Zhao"
]

automation_robotics_control_faculty = [
    "Wayne Book",
    "Ye-Hwa Chen",
    "Stephen Dickerson",
    "Frank Hammond",
    "Shreyas Kousik",
    "Thomas Kurfess",
    "Kok-Meng Lee",
    "Anirban Mazumdar",
    "Ellen Mazumdar",
    "Nader Sadegh",
    "Gregory Sawicki",
    "William Singhose",
    "Maegan Tucker",
    "Jun Ueda",
    "Aaron Young",
    "Fan Zhang",
    "Ye Zhao"
]

bioengineering_faculty = [
    "Cyrus Aidun",
    "Costas Arvanitis",
    "Jaydev Desai",
    "Brandon Dixon",
    "Ross Ethier",
    "Andrei Fedorov",
    "Craig Forest",
    "Andrés García",
    "Rudolph Gleason",
    "Frank Hammond",
    "David Hu",
    "YongTae Kim",
    "Peter Kottke",
    "David Ku",
    "Julien Meaud",
    "Zhanna Nepiyushchikh",
    "Ankur Singh",
    "Stephen Sprigle",
    "Todd Sulchek",
    "Susan Thomas",
    "Lena Ting",
    "Jun Ueda",
    "Levi Wood",
    "Yusuf Yaras",
    "Hong Yeo",
    "Cheng Zhu"
]

engineering_education_faculty = [
    "Brandon Dixon",
    "Jill Fennell",
    "Aldo Ferri",
    "Jonathan Gaines",
    "Peter Griffiths",
    "Amit Jariwala",
    "Julie Linsey",
    "David MacNair",
    "Roxanne Moore",
    "Raghuram Pucha",
    "Carolyn Seepersad",
    "Mike Tinskey",
    "David Torello"
]

cae_and_design_faculty = [
    "Bert Bras",
    "Seung-Kyum Choi",
    "Denis Dorozhkin",
    "Katherine Fu",
    "Amit Jariwala",
    "Roger Jiao",
    "Julie Linsey",
    "Roxanne Moore",
    "Raghuram Pucha",
    "David Rosen",
    "Emily Sanders",
    "Carolyn Seepersad",
    "Suresh Sitaraman",
    "Yan Wang"
]

fluid_mechanics_faculty = [
    "Cyrus Aidun",
    "Alexander Alexeev",
    "Tom Crittenden",
    "Brandon Dixon",
    "Andrei Fedorov",
    "Ari Glezer",
    "David Hu",
    "Suhas Jain",
    "David Ku",
    "Satish Kumar",
    "Peter Loutzenhiser",
    "Ellen Mazumdar",
    "Devesh Ranjan",
    "Marc Smith",
    "Bojan Vukasinovic",
    "Yeung"
]

heat_transfer_combustion_energy_systems_faculty = [
    "Alexander Alexeev",
    "Hailong Chen",
    "Tom Crittenden",
    "Andrei Fedorov",
    "Srinivas Garimella",
    "Caroline Genzale",
    "Mostafa Ghiaasiaan",
    "Ari Glezer",
    "Samuel Graham",
    "Marta Hatzell",
    "Asegun Henry",
    "Yunhyeok Im",
    "Suhas Jain",
    "Ardalan Javadi",
    "Sheldon Jeter",
    "Yogendra Joshi",
    "Peter Kottke",
    "Satish Kumar",
    "Seung Woo Lee",
    "Timothy Lieuwen",
    "Peter Loutzenhiser",
    "Rhett Mayor",
    "Matthew McDowell",
    "Akanksha Menon",
    "Bud Peterson",
    "Devesh Ranjan",
    "Louis Satyanarayan",
    "Marc Smith",
    "Bojan Vukasinovic",
    "Shannon Yee",
    "Zhuomin Zhang",
    "Ben Zinn"
]

manufacturing_faculty = [
    "Nazanin Bassiri-Gharb",
    "Bert Bras",
    "Ye-Hwa Chen",
    "Jonathan Colton",
    "Steven Danyluk",
    "Suman Das",
    "Andrei Fedorov",
    "Tequila Harris",
    "Peter Hesketh",
    "Amit Jariwala",
    "Roger Jiao",
    "Kyriaki Kalaitzidou",
    "Thomas Kurfess",
    "Steven Liang",
    "Rhett Mayor",
    "Leon McGinnis",
    "Shreyes Melkote",
    "Michael Pagan",
    "Raghuram Pucha",
    "David Rosen",
    "Sourabh Saha",
    "Christopher Saldaña",
    "Carolyn Seepersad",
    "Jianjun Shi",
    "Suresh Sitaraman",
    "Aaron Stebner",
    "Jun Ueda",
    "Yan Wang",
    "Chuck Zhang",
    "Min Zhou"
]

mechanics_materials_faculty = [
    "Stephen Antolovich",
    "Antonia Antoniou",
    "Nazanin Bassiri-Gharb",
    "Hailong Chen",
    "Seung-Kyum Choi",
    "Chaitanya Deo",
    "Alper Erturk",
    "Rudolph Gleason",
    "Yuhang Hu",
    "Karl Jacob",
    "Laurence Jacobs",
    "W. Steven Johnson",
    "Kyriaki Kalaitzidou",
    "Surya Kalidindi",
    "Michael Leamy",
    "David McDowell",
    "Matthew McDowell",
    "Julien Meaud",
    "Farrokh Mistree",
    "Richard Neu",
    "Michael Pagan",
    "Etienne Patoor",
    "Olivier Pierron",
    "H. Jerry Qi",
    "David Rosen",
    "Christopher Saldaña",
    "Emily Sanders",
    "Suresh Sitaraman",
    "Aaron Stebner",
    "Naresh Thadhani",
    "Yan Wang",
    "Shuman Xia",
    "Min Zhou",
    "Ting Zhu"
]

medical_physics_faculty = [
    "Said Abdel-Khalik",
    "Steven Biegalski",
    "Chaitanya Deo",
    "Shaheen Dewji",
    "Eric Elder",
    "Anna Erickson",
    "Nolan Hertel",
    "Liyong Lin",
    "Bojan Petrovic",
    "Farzad Rahnema",
    "Chris Wang",
    "Xiaofeng Yang",
    "Fan Zhang",
    "Dingkang Zhang"
]

micro_nano_engineering_faculty = [
    "Alexander Alexeev",
    "Antonia Antoniou",
    "Nazanin Bassiri-Gharb",
    "Hailong Chen",
    "Jonathan Colton",
    "Levent Degertekin",
    "Andrei Fedorov",
    "Samuel Graham",
    "Marta Hatzell",
    "Peter Hesketh",
    "Yunhyeok Im",
    "Yogendra Joshi",
    "YongTae Kim",
    "Seung Woo Lee",
    "Matthew McDowell",
    "Akanksha Menon",
    "Olivier Pierron",
    "Sourabh Saha",
    "Suresh Sitaraman",
    "Vanessa Smet",
    "Todd Sulchek",
    "Shannon Yee",
    "Hong Yeo",
    "Zhuomin Zhang"
]

nre_faculty = [
    "Said Abdel-Khalik",
    "Greeshma Agasthya",
    "Steven Biegalski",
    "Chaitanya Deo",
    "Shaheen Dewji",
    "Anna Erickson",
    "Mostafa Ghiaasiaan",
    "Nolan Hertel",
    "Dan Kotlyar",
    "Jonathon Nye",
    "Bojan Petrovic",
    "Farzad Rahnema",
    "Devesh Ranjan",
    "Weston Stacey",
    "Yuguo Tao",
    "James Wall",
    "Chris Wang",
    "Shannon Yee",
    "Fan Zhang",
    "Dingkang Zhang"
]

tribology_faculty = [
    "Richard Cowan",
    "Steven Danyluk",
    "Itzhak Green",
    "Richard Neu",
    "Richard Salant",
    "Ward Winer"
]


In [31]:
def find_rag_for_professor(prof_name):
    """
    Takes a professor's name in all capital letters and determines which RAG they are part of, if any.
    
    Parameters:
    - prof_name (str): Professor's name in all caps format.
    - rag_faculty_lists (dict): Dictionary where keys are RAG names and values are lists of faculty.
    
    Returns:
    - result (str): The RAGs that the professor is explicitly listed in or a statement if none.
    """
    # Convert professor name to title case
    professor_name = prof_name.title()
    
    # Initialize list for the RAGs that the professor is part of
    rag_memberships = []
    
    if professor_name in acoustics_dynamics_faculty:
        rag_memberships.append("Acoustics & Dynamics")
    if professor_name in ai2me_faculty:
        rag_memberships.append("AI and Informatics for ME (AI2ME)")
    if professor_name in automation_robotics_control_faculty:
        rag_memberships.append("Automation, Robotics, and Control")
    if professor_name in bioengineering_faculty:
        rag_memberships.append("Bioengineering")
    if professor_name in cae_and_design_faculty:
        rag_memberships.append("CAE & Design")
    if professor_name in engineering_education_faculty:
        rag_memberships.append("Engineering Education")
    if professor_name in fluid_mechanics_faculty:
        rag_memberships.append("Fluid Mechanics")
    if professor_name in heat_transfer_combustion_energy_systems_faculty:
        rag_memberships.append("Heat Transfer, Combustion, and Energy Systems")
    if professor_name in manufacturing_faculty:
        rag_memberships.append("Manufacturing")
    if professor_name in mechanics_materials_faculty:
        rag_memberships.append("Mechanics & Materials")
    if professor_name in medical_physics_faculty:
        rag_memberships.append("Medical Physics")
    if professor_name in micro_nano_engineering_faculty:
        rag_memberships.append("Micro and Nano Engineering")
    if professor_name in nre_faculty:
        rag_memberships.append("Nuclear and Radiological Engineering")
    if professor_name in tribology_faculty:
        rag_memberships.append("Tribology")
    
    # Return result based on membership
    if rag_memberships:
        return f"{professor_name} is explicitly listed in the following RAG(s): {', '.join(rag_memberships)}"
    else:
        return f"{professor_name} isn't explicitly in any of the RAGs on the Woodruff School faculty website."

In [32]:
def classify_title(professor_name, department, title):
    rag_prompt = find_rag_for_professor(professor_name)

    messages = [
        {"role": "system", "content": "You are a helpful assistant. You are classifying Georgia Tech research projects based on their titles, professors, and departments."},
        {"role": "user", "content": f"""
        Classify the following project title into one Research Area Group (RAG) and suggest relevant themes. 

        Project Information (Use the provided information to help with classification):
        - Professor: {professor_name}
        - Department: {department}
        - Title: '{title}'
        - {rag_prompt}
        
        Research Area Groups (choose ONLY ONE):
        - Acoustics/Dynamics
        - AI and Informatics for ME (AI2ME)
        - Automation, Robotics, and Control
        - Bioengineering
        - CAE and Design
        - Engineering Education
        - Fluid Mechanics
        - Heat Transfer, Combustion, & Energy Systems
        - Manufacturing
        - Mechanics of Materials
        - Medical Physics
        - Micro and Nano Engineering
        - Nuclear & Radiological Engineering
        - Tribology
        
        Themes (choose ONLY ONE):
        - Sustainability & Energy
        - Health, Water & Agriculture
        - Ocean, Space, Aerospace
        - Design, Materials, Manufacturing
        - Mobility/Automotive/Robotics
        
        Please follow these conditions:
        - Select ONLY ONE option for the RAG.
        - Select ONLY ONE option for the themes.
        - Use only the options provided; do not add or invent any new categories.
        
        Response format:
        - RAG: <One Research Area Group>
        - Themes: <One Theme>
        """}
    ]
    
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    
    # Extract response content
    content = response.choices[0].message.content

    print(content)  # Print the response content for debugging
    
    # Parse the RAG and Themes from the response content using regex
    rag_match = re.search(r'RAG:\s*(.*)', content)
    themes_match = re.search(r'Themes:\s*(.*)', content)

    # Extract RAG and Themes if they match, or set to None if not found
    rag = rag_match.group(1).strip() if rag_match else None
    themes = themes_match.group(1).strip() if themes_match else None
    
    return rag, themes


In [45]:
#  Loop through each row in award_projects_data and apply the classification function
for idx, row in award_projects_data.iterrows():
    try:
        print(f"Processing row {idx + 1}/{len(award_projects_data)}")
        print(f"Project Title: {row['Proj Title']}")

        # Classify project using professor name, department, and project title
        rag, themes = classify_title(row['Formatted Name'], row['Department'], row['Proj Title'])
        
        # Print and store the classification result
        print(f"RAG: {rag}, Themes: {themes}")
        award_projects_data.loc[idx, 'RAG'] = rag
        award_projects_data.loc[idx, 'Themes'] = themes
        
        # Delay to avoid rate limiting
        time.sleep(1)
    
    except Exception as e:
        print(f"Error processing row {idx}: {str(e)}")
        award_projects_data.loc[idx, 'RAG'] = None
        award_projects_data.loc[idx, 'Themes'] = None
        time.sleep(1)

print("\nClassification completed for all rows.")

Processing row 1/1133
Project Title: CAREER:Interfacial Tissue Physics and Wearable Scalp Technologies for Long-
- RAG: Bioengineering
- Themes: Health, Water & Agriculture
RAG: Bioengineering, Themes: Health, Water & Agriculture
Processing row 2/1133
Project Title: DEVELOPING AIR-COOLED PASSIVE TYPE ULTRA-LIGHT FUEL CELL MEA OPERATED WITH
- RAG: Heat Transfer, Combustion, & Energy Systems
- Themes: Sustainability & Energy
RAG: Heat Transfer, Combustion, & Energy Systems, Themes: Sustainability & Energy
Processing row 3/1133
Project Title: SSA WEAV3D INC.
- RAG: Mechanics of Materials
- Themes: Design, Materials, Manufacturing
RAG: Mechanics of Materials, Themes: Design, Materials, Manufacturing
Processing row 4/1133
Project Title: CONSTRUCTION OF A PROTOTYPE 1 GPA VISCOMETER
- RAG: Tribology
- Themes: Design, Materials, Manufacturing
RAG: Tribology, Themes: Design, Materials, Manufacturing
Processing row 5/1133
Project Title: DEVELOPMENT OF A HIGH PRESSURE VISCOMETER (COOPERATIVE AGRE

In [46]:
award_projects_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1133 entries, 0 to 1132
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Award Type Desc             1132 non-null   object 
 1   Co Pdpi Names               238 non-null    object 
 2   Contract Type Desc          1128 non-null   object 
 3   Department                  1133 non-null   object 
 4   Doch Keywords               302 non-null    object 
 5   FY Year                     1133 non-null   object 
 6   Pdpi Name                   1133 non-null   object 
 7   Pdpi Org Name               1133 non-null   object 
 8   Proj Status Desc            1132 non-null   object 
 9   Proj Title                  1133 non-null   object 
 10  Project Type Desc           1132 non-null   object 
 11  Ps Project Number           1129 non-null   object 
 12  Sponsor Business Type Desc  1133 non-null   object 
 13  Sponsor Name                1133 non-n

In [60]:
# Save the updated dataframe to a new excel file
output_file_path = r'C:\Users\donav\OneDrive\Desktop\Gatech\Research\GRA Ford Research\ME_Woodruff_Classifier\data\award_data_with_classification_3.xlsx'

award_projects_data.to_excel(output_file_path, index=False)

In [59]:
# Check the unique values in the RAG and Themes columns
print(award_projects_data['RAG'].unique())
print(len(award_projects_data['RAG'].unique()))
print(award_projects_data['Themes'].unique())
print(len(award_projects_data['Themes'].unique()))

['Bioengineering' 'Heat Transfer, Combustion, & Energy Systems'
 'Mechanics of Materials' 'Tribology' 'Micro and Nano Engineering'
 'Acoustics/Dynamics' 'Nuclear & Radiological Engineering' 'Manufacturing'
 'CAE and Design' 'Engineering Education' 'Fluid Mechanics'
 'Medical Physics' 'AI and Informatics for ME (AI2ME)'
 'Automation, Robotics, and Control']
14
['Health, Water & Agriculture' 'Sustainability & Energy'
 'Design, Materials, Manufacturing' 'Ocean, Space, Aerospace'
 'Mobility/Automotive/Robotics']
5
